# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
# !pip install selenium
# além de instalar o selenium, instalar o driver do seu navegador
# Chrome -> chromedriver; Firefox -> geckodriver
# https://chromedriver.chromium.org/downloads
# pegar a versão de acordo com a versão do seu Chrome e instalar no local em que o Python vai rodar

In [2]:
from selenium import webdriver # permite criar o navegador
from selenium.webdriver.common.keys import Keys # permite você escrever no navegador
from selenium.webdriver.common.by import By # permite você selecionar itens no navegador

# para rodar o chrome em 2º plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True 
# navegador = webdriver.Chrome(options=chrome_options)

# abrir o navegador
navegador = webdriver.Chrome()
# caso queira deixar na mesma pasta do seu código
# navegador = webdriver.Chrome("chromedriver.exe")

navegador.get("https://www.google.com.br/")

# Olhar os XPaths de cada elemento que você quer utilizar com o F12/Inspecionar Elemento da página
# Para pegar alguma informação de um elemento, olhar nos atributos o que te interessa
# Passo 1: Pegar a cotação do dólar
navegador.find_element("xpath",
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element("xpath",
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 
print(cotacao_dolar)

# Passo 2: Pegar a cotação do euro
navegador.get("https://www.google.com.br/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 3: Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()

5.204403
5.071690723000001
284.63


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [3]:
# Passo 4: Importar a lista de produtos
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [4]:
# Passo 5: Recalcular o preço de cada produto
# Preciso atualizar a coluna de cotação. Localizo os lugares com o .loc e atualizo
# tabela.loc[linha, coluna] = novo valor

# nas linhas em que o valor da coluna "Moeda" == Dólar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
# nas linhas em que o valor da coluna "Moeda" == Euro
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
# nas linhas em que o valor da coluna "Moeda" == Ouro
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar a coluna de preço de compra (preço original * cotação atual)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar a coluna de preço final (preço de compra * margem de lucro)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

# atualizar a formatação da última coluna, caso queira
tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.204403,5204.350956,1.40,R$7286.09
1,Carro Renault,4500.00,Euro,5.071691,22822.608254,2.00,R$45645.22
2,Notebook Dell,899.99,Dólar,5.204403,4683.910656,1.70,R$7962.65
3,IPhone,799.00,Dólar,5.204403,4158.317997,1.70,R$7069.14
4,Carro Fiat,3000.00,Euro,5.071691,15215.072169,1.90,R$28908.64
5,Celular Xiaomi,480.48,Dólar,5.204403,2500.611553,2.00,R$5001.22
6,Joia 20g,20.00,Ouro,284.630000,5692.600000,1.15,R$6546.49


### Agora vamos exportar a nova base de preços atualizada

In [5]:
# Passo 6: Salvar os novos preços dos produtos
tabela.to_excel("ProdutosAtt.xlsx", index=False) # index = False pra não colocar os índices 0..6 na base de dados (irrelevante)